<h6><center>Big Data Algorithms Techniques & Platforms</center></h6>

<h1>
<hr style=" border:none; height:3px;">
<center>Assignment 1: Introduction to MapReduce</center>
<hr style=" border:none; height:3px;">
</h1>


# <strong> Important </strong>

For this assignment, as you did in the first lab, do not use the native fuctions <code>map()</code> and <code>reduce()</code> of Python.


<hr>



## Flights and Airports Data

For this assignment you are going to analyze a dataset that will include flight data.

The dataset comes from <a href="https://www.kaggle.com/flashgordon/usa-airport-dataset">Kaggle</a> and is presented here as an extract called <code>extract_flights.csv</code>

The dataset is in a <code>.csv</code> file. Each line of the file represents a different flight. The data collected contain:


<code>Origin_airport</code>: Three letter airport code of the origin airport </br>
<code>Destination_airport</code>: Three letter airport code of the destination airport</br>
<code>Origin_city</code>: Origin city name</br>
<code>Destination_city</code>: Destination city name</br>
<code>Passengers</code>: Number of passengers transported from origin to destination</br>
<code>Seats</code>: Number of seats available on flights from origin to destination</br>
<code>Flights</code>: Number of flights between origin and destination (multiple records for one month, many with flights > 1)</br>
<code>Distance</code>: Distance (to nearest mile) flown between origin and destination</br>
<code>Fly_date</code>: The date (yyyymm) of flight</br>
<code>Origin_population</code>: Origin city's population as reported by US Census</br>
<code>Destination_population</code>: Destination city's population as reported by US Census</br>


Two files are available in the folder:

<ul>
  <li> <code> small_file.csv</code> </li>
  <li> <code> medium_file.csv</code> </li>
</ul>

For more complex analysis/statistics you can also download the original file available on kaggle.



### Helper function ... just in case

Here is a function that you can modify, reuse, not-use for giving you an idea about how to read a <code>.csv</code> file and remove the header.

In [ ]:
# Here a simple function that reads a .csv file, removes the header,
# and writes the output on a .txt file having the same name of the input file + the keyword "no_header"
# input: a csv file
def remove_header(filename) :
  with open(filename + ".csv",'r') as f:
    with open(filename + "no_header.txt",'w') as f1:
        next(f) # skip header line
        for line in f:
            f1.write(line)
  return f1

remove_header('/content/small_file')
print("file correctly saved")


file correctly saved


### <strong> Exercise 1 - Almost-empty flights</strong> 
#### <strong> 4 points </strong>
Describe and define a MapReduce procedure that gives as output the number of flights that departed with 10% of capacity maximum (i.e. empty, 1%, 6%, etc.) . 

The output can be of any form you like. 

You can use any data structure you want to support your implementation.




In [10]:
# ASSUMPTION 1: 0 seats flights are not passenger flights and 
#               are removed from the analysis data.
# ASSUMPTION 2: for a route in a month, if the proportion of passengers and 
#               seats is less or equal than 10% then all the flights are
#               assumed to be almost empty 

# MAIN PROCESS:
# file reader       -> dataframe using pandas and return a list.
# map function      -> list of [(a, n), (b, n), ...] a: above 10% below 10%. 
# shuffle function  -> list of [[(a, [n,m,..]),(b, [n,m,..])].
# reduce function   -> list of [(a, total above), (b, total below)].
# _____
# detailed documentation of each function below:

import pandas as pd

# prerrequisites: this reader uses pandas library to read csv and choose
#                 the relevant data.
# input:  the finename to be read with its route.
# logic:  this function reads the csv file with pandas, calculates the
#         proportion and returns it as a list.
# output: a list containing the percentage of capacity that was used from
#         each flight.
def file_reader_1(filename):
  df = pd.read_csv(filename)
  df = df[df['Seats']>0] 
  df['proportion'] = df['Passengers']/df['Seats']
  df = df[['proportion','Flights']]
  response = df.values.tolist()

  return response

# input:  the list with the proportion of the flights.
# logic:  this function will evaluate each proportion, if it is above 10% 
#         it will assign the tuple (a, n), when it is below or equal to 10% 
#         it will assign the tuple (b, n) to that flight. Where n is the number
#         of flights for a specific route during a specific month.
# output: list of tuples with (a, n) or (b, n) depending on the logic
#         described.
def map_e_flights_1(array):
  pairs = []
  for l in array :
    if (l[0] > 0.1 ) :
      pairs.append(('a',l[1]))
    else :
      pairs.append(('b',l[1]))
  return (pairs)

# input:  the list with the tuples (a, n) or (b, n) of the flights.
# logic:  this function will use the input and organize it in two lists,
#         the one with above (a, [n,m,o,...]) and below (b, [n,m,o,....]]).
# output: two lists with the tuples organized.
def shuffle_e_flights_1(array):
  above = []
  below = []
  for flight in array:
    if flight[0] == 'a':
      above.append(flight[1])
    else:
      below.append(flight[1])
  shuffled = []
  shuffled.append(('a',above))
  shuffled.append(('b',below))
  return shuffled

# input:  the list with the tuples (a, [n,m,o,...]) or (b, [n,m,o,...])
#         organized.
# logic:  this function will use the input and sum it up.
# output: the final answer.
def reduce_e_flights_1(array):
  above = ('a', sum(array[0][1]))
  below = ('b', sum(array[1][1]))
  return [above, below]

# Function exectution
flights = file_reader_1('/content/small_file.csv')
mapped = map_e_flights_1(flights)
shuffled = shuffle_e_flights_1(mapped)
reduced = reduce_e_flights_1(shuffled)
reduced


[('a', 138748.0), ('b', 1267.0)]

Look at your implementation and think about your program. Is there any option to add a `combine` operation?  If yes describe your new approach and provide an improved version of your code.

Describe in detail your procedure before the code (in the comment lines).

In [11]:
# The previous funtions evaluate the whole input file in one list
# instead of using partial data, this new functions will take
# a portion of the data in a set of list, and use a combine function
# to have partial calculations.

# ASSUMPTION 1: 0 seats flights are not passenger flights and 
#               are removed from the analysis data.
# ASSUMPTION 2: for a route in a month, if the proportion of passengers and 
#               seats is less or equal than 10% then all the flights are
#               assumed to be almost empty 

# MAIN PROCESS:
# file reader       -> dataframe using pandas and return a list.
# map function      -> list of lists of size 20 with [(a, n), (b, n), ...] 
#                      a: above 10% below 10%.
# combine function  -> sums the partial result of each sublist [a, sum(n)],
#                      [b, sum(n)].
# shuffle function  -> list of [[(a, [n,m,..]),(b, [n,m,..])].
# reduce function   -> list of [(a, total above), (b, total below)].
# _____
# detailed documentation of each function below:


import pandas as pd

# prerrequisites: this reader uses pandas package to read csv and choose
#                 the relevant data.
# input:  the finename to be read with its route.
# logic:  this function reads the csv file with pandas, calculates the
#         proportion and returns it as a list of lists of size 20. this change
#         was done to be able to use a combine function 
# output: a list containing the percentage of capacity that was used from
#         each flight with size 20.
def file_reader_2(filename):
  df = pd.read_csv(filename)
  df = df[df['Seats']>0]
  df['proportion'] = df['Passengers']/df['Seats']
  df = df[['proportion','Flights']]
  full_list = df.values.tolist()
  
  response = []
  partial_response = []
  for i in range (0, len(full_list)):
      if i%20 == 19:
        partial_response.append(full_list[i])
        response.append(partial_response)
        partial_response = []
      else:
        partial_response.append(full_list[i])
  response.append(partial_response)

  return response

# input:  the list with the proportion of the flights.
# logic:  this function will evaluate each proportion, if it is above 10% 
#         it will assign the tuple (a, n), when it is below or equal to 10% 
#         it will assign the tuple (b, n) to that flight.
# output: list of lists with tuples with (a, n) or (b, n) depending on 
#         the logic described.
def map_e_flights_2(array):
  pairs = []
  for l in array :
    respart = []
    for flight in l:
      if (flight[0] > 0.1 ) :
        respart.append(('a',flight[1]))
      else :
        respart.append(('b',flight[1]))
    pairs.append(respart)
  
  return (pairs)

# input:  the list with the proportion of the flights.
# logic:  this function will sum the number of above and below within
#         each list.
# output: list of tuples with (a, n) or (b, m) depending on the count of each
#         sub-list.
def combine_e_flights_2(array):
  pairs = []
  for l in array :
    a = 0
    b = 0
    for flight in l:
      if flight[0] == 'a':
        a += flight[1]
      elif flight[0] == 'b':
        b += flight[1]
    pairs.append([('a',a),('b',b)])
  return (pairs)

# input:  the list with the tuples ('a', n) or ('b', m) of the flights.
# logic:  this function will use the input and organize it in two lists
#         shuffled for each option (a = above) or (b = below)
# output: two lists with the sum of each sublist (a,[20,10,...]) and 
#         (b,[2,4,5,...]
def shuffle_e_flights_2(array):
  above = []
  below = []
  shuffled = []
  for col in array: 
    above.append(col[0][1])
    below.append(col[1][1])
  shuffled.append(('a', above))
  shuffled.append(('b', below))
  return shuffled

# input:  the list with the tuples (a,[20,10,...]) and (b,[2,4,5,...]).
# logic:  this function will use the input and sum it up.
# output: the final answer.
def reduce_e_flights_2(array):
  above = ('a', sum(array[0][1]))
  below = ('b', sum(array[1][1]))
  return [above, below]

# Function exectution
flights  = file_reader_2('/content/small_file.csv')
mapped   = map_e_flights_2(flights)
combined = combine_e_flights_2(mapped)
shuffled = shuffle_e_flights_2(combined)
reduced   = reduce_e_flights_2(shuffled)
reduced


[('a', 138748.0), ('b', 1267.0)]

### <strong> Exercise 2 - Top five destination airports </strong>
### <strong> 4 points </strong>

Provide now a function that lists the top five destination <strong>airports</strong>: the ones that have the highest number of incoming flights. Implement an algorithm that uses the MapReduce procedure.



In [12]:
# ASSUMPTION 1: 0 seats flights are not passenger flights and 
#               are removed from the analysis data.

# MAIN PROCESS:
# file reader       -> dataframe using pandas and return a list.
# map function      -> list with [(destA, n), (destB, n), ...] with the
#                      number of flights.
# shuffle function  -> list of [[(destA, [n,m,..]),(destB, [n,m,..])].
# reduce function   -> list of [(destA, total flights), (destB, total flights)
#                               (destC, total flights),... ].
# _____
# detailed documentation of each function below:

import pandas as pd

# prerrequisites: this reader uses pandas package to read csv and choose
#                 the relevant data.
# input:  the finename to be read with its route.
# logic:  this function reads the csv file with pandas, calculates the
#         proportion and returns it as a list with the airport and # of flights.
# output: a list containing the destination airport and number of flights.
def file_reader_3(filename):
  df = pd.read_csv(filename)
  df = df[df['Seats']>0]
  df = df[['Destination_airport','Flights']]
  response = df.values.tolist()
  return response

# input:  the list with the flights per destination airport.
# logic:  this function will map each value from the input and map it as a
#         tuple (destination, x) where x is the number of flights
# output: list of tuples with (destination, x).
def map_top_flights(array):
  pairs = []
  for flight in array :
    pairs.append((flight[0],flight[1]))
    
  return (pairs)

# input:  the list with the flights per destination airport.
# logic:  this function will shuffle each value from the input and map it as a
#         tuple (destination, [x,y,z,...]) where x, y, z  are the number of
#         flights.
# output: list of tuples with (destination, [x,y,z,...]).
def shuffle_top_flights(array):
  shuffled_list = []

  lista = list(set([item[0] for item in array]))

  for element in lista:
    numbers = []
    for value in array:
      if element == value[0]:
        numbers.append(value[1])
    shuffled_list.append((element,numbers))
  
  return shuffled_list

# input:  the list with shufled per destination airport.
# logic:  this function will map each value from the input and reduce the values
#         by summing them up. After that it will reorder and return the top 5
# output: list of top 5 destination airports (destination, x), ... .
def reduce_top_flights(array):
  reduced_list = []
  for value in array:
    reduced_list.append((value[0],sum(value[1])))
  
  values = sorted(list(set([item[1] for item in reduced_list])),reverse=True)

  reduced_list_ordered = []

  for value in values:
    if len(reduced_list_ordered) < 5:
      for airport in reduced_list:
        if value == airport[1]:
          reduced_list_ordered.append((airport[0],airport[1]))

  return reduced_list_ordered[0:5]

# Function exectution
flights  = file_reader_3('/content/small_file.csv')
mapped   = map_top_flights(flights)
shuffled = shuffle_top_flights(mapped)
reduced  = reduce_top_flights(shuffled)
reduced

[('RDM', 103901), ('EKO', 25711), ('ERI', 7101), ('WDG', 3295), ('END', 6)]

### <strong> Exercise 3 - Top 5 destination cities </strong>
#### <strong>  2 points </strong>

Try to reuse the code you run before and define a function that lists the top five destination <strong>cities</strong>: the ones that have the highest number of incoming flights. Implement an algorithm that uses the MapReduce procedure.


In [13]:
# ASSUMPTION 1: 0 seats flights are not passenger flights and 
#               are removed from the analysis data.

# MAIN PROCESS:
# file reader       -> dataframe using pandas and return a list.
# map function      -> list with [(destA, n), (destB, n), ...] with the
#                      number of flights.
# shuffle function  -> list of [[(destA, [n,m,..]),(destB, [n,m,..])].
# reduce function   -> list of [(destA, total flights), (destB, total flights)
#                               (destC, total flights),... ].
# _____
# detailed documentation of each function below:

import pandas as pd

# prerrequisites: this reader uses pandas package to read csv and choose
#                 the relevant data.
# input:  the finename to be read with its route.
# logic:  this function reads the csv file with pandas, calculates the
#         proportion and returns it as a list with the airport and # of flights.
# output: a list containing the destination airport and number of flights.
def file_reader_4(filename):
  df = pd.read_csv(filename)
  df = df[df['Seats']>0]
  df = df[['Destination_city','Flights']]
  response = df.values.tolist()
  return response


# input:  the list with the flights per destination city.
# logic:  this function will map each value from the input and map it as a
#         tuple (destination, x) where x is the number of flights
# output: list of tuples with (destination, x).
def map_top_cities(array):
  pairs = []
  for flight in array :
    pairs.append((flight[0],flight[1]))
    
  return (pairs)

# input:  the list with the flights per destination city.
# logic:  this function will shuffle each value from the input and map it as a
#         tuple (destination, [x,y,z,...]) where x, y, z  are the number of
#         flights.
# output: list of tuples with (destination, [x,y,z,...]).
def shuffle_top_cities(array):
  shuffled_list = []

  lista = list(set([item[0] for item in array]))

  for element in lista:
    numbers = []
    for value in array:
      if element == value[0]:
        numbers.append(value[1])
    shuffled_list.append((element,numbers))
  
  return shuffled_list

# input:  the list with shufled per destination city.
# logic:  this function will map each value from the input and reduce the values
#         by summing them up. After that it will reorder and return the top 5
# output: list of top 5 destination cities (destination, total).

def reduce_top_cities(array):
  reduced_list = []
  for value in array:
    reduced_list.append((value[0],sum(value[1])))
  
  values = sorted(list(set([item[1] for item in reduced_list])),reverse=True)

  reduced_list_ordered = []

  for value in values:
    if len(reduced_list_ordered) < 5:
      for airport in reduced_list:
        if value == airport[1]:
          reduced_list_ordered.append((airport[0],airport[1]))

  return reduced_list_ordered[0:5]

# Function exectution
flights  = file_reader_4('/content/small_file.csv')
mapped   = map_top_cities(flights)
shuffled = shuffle_top_cities(mapped)
reduced  = reduce_top_cities(shuffled)
reduced

[('Bend, OR', 103901),
 ('Elko, NV', 25711),
 ('Erie, PA', 7101),
 ('Enid, OK', 3301),
 ('Ames, IA', 1)]

## Statistics on flights

<p align="justify">
<font size="3">
Now we want to run some more complex analysis on the flights. 
</font>
</p>





### <strong> Exercise 4 - Top five connections by month</strong>
#### <strong> 4 points </strong>

Try to reuse the code you run before and define now a function that lists the top five connections by each month: the top five pairs of cities that have the most number of connections (the function takes into account the flights from A to B and from B to A by month/year). Implement an algorithm that uses the MapReduce procedure.

In [14]:
# ASSUMPTION 1: 0 seats flights are not passenger flights and 
#               are removed from the analysis data.
# ASSUMPTION 2: a connection is assumed to be a route A->B and B->A. The 
#               number of flights per connection per month for A->B is the sum
#               of flights(A->B) and flights(B->A).

# MAIN PROCESS:
# file reader       -> dataframe using pandas and return a list.
# map function      -> list with [(date_1, [(connectionA, flights),
#                                           (connectionB, flights), ...]),
#                                 (date_2, [(connectionA, flights),
#                                           (connectionB, flights), ...]), ...]        
# shuffle function  -> list with [(date_1, [(connectionA, [flightA, flightB,...]),
#                                           (connectionB, [flightA, flightB,...]),
#                                           ...]),
#                                 (date_2, [(connectionA, [flightA, flightB,...]),
#                                           (connectionB, [flightA, flightB,...]),
#                                           ...]), ...]   
# reduce function   -> list with [(date_1, [(connectionA, Total_flights),
#                                           (connectionB, Total_flights), ...]),
#                                 (date_2, [(connectionA, Total_flights),
#                                           (connectionB, Total_flights), ...]),
#                                           ...]   
# _____
# detailed documentation of each function below:

import pandas as pd
import numpy as np

# prerrequisites: this reader uses pandas package to read csv and choose
#                 the relevant data.
# input:  the finename to be read with its route.
# logic:  this function reads the csv file with pandas, returns it as a list 
#         with the month, origin, destination and number of flights.
# output: a list containing the destination airport and number of flights.
def file_reader_5(filename):
  df = pd.read_csv(filename)
  df = df[df['Seats']>0]
  df = df[['Fly_date','Origin_city','Destination_city','Flights']]
  df['Origin'] = np.where(df['Origin_city'] < df['Destination_city'], df['Origin_city'], df['Destination_city'])
  df['Destination'] = np.where(df['Origin_city'] < df['Destination_city'], df['Destination_city'], df['Origin_city'])
  df = df[['Fly_date','Origin','Destination','Flights']]

  response = df.values.tolist()
  return response

# input:  the list with the date, connection, and number of flights data.
# logic:  this function maps the data as (date,(connection, flights)) tuples.
# output: a list containing the tuples described in the logic.
def map_top_connections(array):
  tuples = []
  for flight in array:
    tuples.append((flight[0],((flight[1]+' <--> '+flight[2]),flight[3])))
  return (tuples)

# input:  the list with the tuples (date,(connection, flights)).
# logic:  this function transforms the data as 
#         (date,(connection, [a, b])) where [a, b] is the number of flights for
#         the combination of date an connection.
# output: a list containing the tuples described in the logic.
def shuffle_top_connections(array):
  combinations = []
  for flight in array:
    combinations.append((flight[0], flight[1][0]))
  
  combinations = list(set(combinations))

  shuffled = []

  for combination in combinations:
    comb = [] 
    for flight in array:
      if combination[0] == flight[0] and combination[1] == flight[1][0]:
        comb.append(flight[1][1])
    shuffled.append((combination[0],combination[1],comb))

  return shuffled

# input:  the list with the tuples (date,(connection, [a, b])).
# logic:  this function does the following:
#         1. reduces the input list
#         2. sorts the top 5 connections by date
#         3. organizes the date in the form
#            [(date1, [top 5 connections])
#             (date2, [top 5 connections]), ...]
#            each [top 5 connections] has the following structure:
#            [(top 1 connection, number of flights),
#             (top 2 connection, number of flights), ...]
# output: a list containing the tuples described in the logic.

def reduce_top_connections(array):
  reduced_1 = []
  dates = []
  for tuples in array:
    reduced_1.append((tuples[0],tuples[1],sum(tuples[2])))
    dates.append(tuples[0])

  dates = sorted(list(set(dates)))
  conns = []
  routes = []

  for date in dates:
    routes = []
    for reduced in reduced_1:
      if(date == reduced[0]):
        routes.append((reduced[1],reduced[2]))
    conns.append((date,routes))

  definitive = []
  lista = []

  for conn in conns:
    lista = sorted(conn[1], key = lambda x: x[1], reverse=True)[0:5]
    definitive.append((conn[0],lista))
  
  return definitive


# Function exectution
flights  = file_reader_5('/content/small_file.csv')
mapped   = map_top_connections(flights)
shuffled = shuffle_top_connections(mapped)
reduced = reduce_top_connections(shuffled)
reduced



[('1990-01-01',
  [('Bend, OR <--> Portland, OR', 184),
   ('Erie, PA <--> Pittsburgh, PA', 150),
   ('Bend, OR <--> San Francisco, CA', 86),
   ('Bend, OR <--> Klamath Falls, OR', 46),
   ('Erie, PA <--> Ithaca, NY', 30)]),
 ('1990-02-01',
  [('Bend, OR <--> Portland, OR', 176),
   ('Erie, PA <--> Pittsburgh, PA', 128),
   ('Bend, OR <--> San Francisco, CA', 83),
   ('Bend, OR <--> Klamath Falls, OR', 38),
   ('Erie, PA <--> Ithaca, NY', 24)]),
 ('1990-03-01',
  [('Bend, OR <--> Portland, OR', 203),
   ('Erie, PA <--> Pittsburgh, PA', 146),
   ('Bend, OR <--> San Francisco, CA', 92),
   ('Bend, OR <--> Klamath Falls, OR', 51),
   ('Erie, PA <--> Ithaca, NY', 29)]),
 ('1990-04-01',
  [('Bend, OR <--> Portland, OR', 197),
   ('Erie, PA <--> Pittsburgh, PA', 144),
   ('Bend, OR <--> San Francisco, CA', 83),
   ('Bend, OR <--> Klamath Falls, OR', 49),
   ('Erie, PA <--> Ithaca, NY', 28)]),
 ('1990-05-01',
  [('Bend, OR <--> Portland, OR', 205),
   ('Erie, PA <--> Pittsburgh, PA', 151),
  

Look at your code. Can you use a <code>combine()</code> operation? 

If you cannot directly think about how you can represent the key-value pairs in a way that allows you to take advantage of a combine.

In [15]:
# ASSUMPTION 1: 0 seats flights are not passenger flights and 
#               are removed from the analysis data.
# ASSUMPTION 2: a connection is assumed to be a route A->B and B->A. The 
#               number of flights per connection per month for A->B is the sum
#               of flights(A->B) and flights(B->A).

# MAIN PROCESS:
# file reader       -> dataframe using pandas and return a list.
# map function      -> list with [(date_1, (connectionA, flight),
#                                 (date_2, (connectionA, flight),
#                                 (date_3, (connectionB, flights), ...]        
# combine function  -> list with [(date_1, [(connectionA, flightA,
#                                           (connectionB, flightB,...]),
#                                           ...]),
#                                 (date_2, [(connectionA, flightA,
#                                           (connectionB, flightB,...]),
#                                           ...]),
# shuffle function  -> list with [(date_1, [(connectionA, [flightA, flightB,...]),
#                                           (connectionB, [flightA, flightB,...]),
#                                           ...]),
#                                 (date_2, [(connectionA, [flightA, flightB,...]),
#                                           (connectionB, [flightA, flightB,...]),
#                                           ...]), ...]   
# reduce function   -> list with [(date_1, [(connectionA, Total_flights),
#                                           (connectionB, Total_flights), ...]),
#                                 (date_2, [(connectionA, Total_flights),
#                                           (connectionB, Total_flights), ...]),
#                                           ...]   
# _____
# detailed documentation of each function below:

import pandas as pd
import numpy as np

# prerrequisites: this reader uses pandas package to read csv and choose
#                 the relevant data.
# input:  the finename to be read with its route.
# logic:  this function reads the csv file with pandas, returns it as a list 
#         with the month, origin, destination and number of flights.
# output: a list containing the destination airport and number of flights.
def file_reader_6(filename):
  df = pd.read_csv(filename)
  df = df[df['Seats']>0]
  df = df[['Fly_date','Origin_city','Destination_city','Flights']]
  df['Origin'] = np.where(df['Origin_city'] < df['Destination_city'], df['Origin_city'], df['Destination_city'])
  df['Destination'] = np.where(df['Origin_city'] < df['Destination_city'], df['Destination_city'], df['Origin_city'])
  df = df[['Fly_date','Origin','Destination','Flights']]

  response = df.values.tolist()
  return response

# input:  the list with the date, connection, and number of flights data.
# logic:  this function maps the data as (date,(connection, flights)) tuples.
# output: a list containing the tuples described in the logic.
def map_top_connections_2(array):
  tuples = []
  for flight in array:
    tuples.append((flight[0],((flight[1]+' <--> '+flight[2]),flight[3])))
  return (tuples)

# input:  the list with the date, connection, and number of flights data.
# logic:  this function maps the data as [(date_1, [(connectionA, flightA,
#                                           (connectionB, flightB,...]),
#                                           ...]),
#                                 (date_2, [(connectionA, flightA,
#                                           (connectionB, flightB,...]),
#                                           ...]),
# output: a list containing the tuples described in the logic.
def combine_top_connections_2(array):
  combinations = []
  for flight in array:
    combinations.append((flight[0], flight[1][0]))
  
  combinations = list(set(combinations))

  combined = []

  for combination in combinations:
    comb = [] 
    for flight in array:
      if combination[0] == flight[0] and combination[1] == flight[1][0]:
        comb.append(flight[1][1])
    combined.append((combination[0],combination[1],comb))
    
  combined_def = []
  
  for tuples in combined:
    combined_def.append((tuples[0],tuples[1],sum(tuples[2])))

  
  return combined_def

# input:  the list with the tuples [(date_1, [(connectionA, flightA,
#                                           (connectionB, flightB,...]),
#                                           ...]),
#                                 (date_2, [(connectionA, flightA,
#                                           (connectionB, flightB,...]),
#                                           ...]),
# logic:  this function transforms the data as 
#         (date,(connection, [a, b])) where [a, b] is the number of flights for
#         the combination of date an connection.
# output: a list containing the tuples described in the logic.
def shuffle_top_connections_2(array):
  
  dates = []
  for values in array:
    dates.append(values[0])
  
  dates = list(set(dates))
  dates = sorted(dates)

  conns = []
  routes = []

  for date in dates:
    routes = []
    for reduced in array:
      if(date == reduced[0]):
        routes.append((reduced[1],reduced[2]))
    conns.append((date,routes))

  return conns

# input:  the list with the tuples (date,(connection, [a, b])).
# logic:  this function does the following:
#         1. reduces the input list
#         2. sorts the top 5 connections by date
#         3. organizes the date in the form
#            [(date1, [top 5 connections])
#             (date2, [top 5 connections]), ...]
#            each [top 5 connections] has the following structure:
#            [(top 1 connection, number of flights),
#             (top 2 connection, number of flights), ...]
# output: a list containing the tuples described in the logic.

def reduce_top_connections_2(array):

  definitive = []
  lista = []

  for element in array:
    lista = sorted(element[1], key = lambda x: x[1], reverse=True)[0:5]
    definitive.append((element[0],lista))
  
  return definitive


# Function exectution
flights  = file_reader_6('/content/small_file.csv')
mapped   = map_top_connections_2(flights)
combined = combine_top_connections_2(mapped)
shuffled = shuffle_top_connections_2(combined)
reduced = reduce_top_connections_2(shuffled)
reduced


[('1990-01-01',
  [('Bend, OR <--> Portland, OR', 184),
   ('Erie, PA <--> Pittsburgh, PA', 150),
   ('Bend, OR <--> San Francisco, CA', 86),
   ('Bend, OR <--> Klamath Falls, OR', 46),
   ('Erie, PA <--> Ithaca, NY', 30)]),
 ('1990-02-01',
  [('Bend, OR <--> Portland, OR', 176),
   ('Erie, PA <--> Pittsburgh, PA', 128),
   ('Bend, OR <--> San Francisco, CA', 83),
   ('Bend, OR <--> Klamath Falls, OR', 38),
   ('Erie, PA <--> Ithaca, NY', 24)]),
 ('1990-03-01',
  [('Bend, OR <--> Portland, OR', 203),
   ('Erie, PA <--> Pittsburgh, PA', 146),
   ('Bend, OR <--> San Francisco, CA', 92),
   ('Bend, OR <--> Klamath Falls, OR', 51),
   ('Erie, PA <--> Ithaca, NY', 29)]),
 ('1990-04-01',
  [('Bend, OR <--> Portland, OR', 197),
   ('Erie, PA <--> Pittsburgh, PA', 144),
   ('Bend, OR <--> San Francisco, CA', 83),
   ('Bend, OR <--> Klamath Falls, OR', 49),
   ('Erie, PA <--> Ithaca, NY', 28)]),
 ('1990-05-01',
  [('Bend, OR <--> Portland, OR', 205),
   ('Erie, PA <--> Pittsburgh, PA', 151),
  

### <strong> Exercise 5 - Number of full flights</strong>
#### <strong> 2 points </strong>
<p align="justify">
<font size="3">
Describe and implement an algorithm that, following MapReduce procedure, shows how many full flights have departed. This exercise gives you an idea about how many times you can re-use code in MapReduce with minimum effort for repetitive analysis.
</font>
</p>



In [16]:
# ASSUMPTION 1: 0 seats flights are not passenger flights and 
#               are removed from the analysis data.
# ASSUMPTION 2: for a route in a month, only if the proportion of passengers and 
#               seats is 100% then all the flights are 100% otherwise there are
#               not full flights.

# MAIN PROCESS:
# file reader       -> dataframe using pandas and return a list.
# map function      -> list of [(yes, n), (no, n), ...] 
# shuffle function  -> list of [[(yes, [n,m,..]),(no, [n,m,..])].
# reduce function   -> list of [(yes, total above), (no, total below)].
# _____
# detailed documentation of each function below:

import pandas as pd

# prerrequisites: this reader uses pandas library to read csv and choose
#                 the relevant data.
# input:  the finename to be read with its route.
# logic:  this function reads the csv file with pandas, calculates the
#         proportion and returns it as a list.
# output: a list containing the percentage of capacity that was used from
#         each flight.
def file_reader_7(filename):
  df = pd.read_csv(filename)
  df = df[df['Seats']>0] 
  df['proportion'] = df['Passengers']/df['Seats']
  df = df[['proportion','Flights']]
  response = df.values.tolist()

  return response

# input:  the list with the proportion of the flights.
# logic:  this function will evaluate each proportion is 1
#         it will assign the tuple (yes, 1), when it is 100% 
#         it will assign the tuple (no, 1) when it is not 100%.
# output: list of tuples with (yes, 1) or (no, 1) depending on the logic
#         described.
def map_c_flights(array):
  pairs = []
  for l in array :
    if (l[0] == 1 ) :
      pairs.append(('yes',l[1]))
    else :
      pairs.append(('no',l[1]))
  return (pairs)

# input:  the list with the tuples (yes, n) or (no, m) of the flights.
# logic:  this function will use the input and organize it in two lists,
#         the one with above (yes, [n,m,o]) and below (no, [n,m,o]).
# output: two lists with the tuples organized.
def shuffle_c_flights(array):
  above = []
  below = []
  for flight in array:
    if flight[0] == 'yes':
      above.append(flight[1])
    else:
      below.append(flight[1])
  shuffled = []
  shuffled.append(('yes',above))
  shuffled.append(('no',below))
  return shuffled

# input:  the list with the tuples (yes, [n,m,o]) or (b, [n,m,o]) organized.
# logic:  this function will use the input and sum it up.
# output: the final answer.
def reduce_c_flights(array):
  above = ('yes', sum(array[0][1]))
  below = ('no', sum(array[1][1]))
  return [above, below]

# Function exectution
flights = file_reader_7('/content/small_file.csv')
mapped = map_c_flights(flights)
shuffled = shuffle_c_flights(mapped)
reduced = reduce_c_flights(shuffled)
reduced


[('yes', 554.0), ('no', 139461.0)]


### <strong> Exercise 6 -  Average/percentage </strong>
#### <strong> 4 points </strong>

<p align="justify">
<font size="3">
Describe and implement a MapReduce procedure that gives, for each city, the percentage/average, of full flights that have departed.

Notice that this exercise shares some similarities with one of the previous exercises. Think how and if you can modify (generalize) one of the functions already implemented before. 
</font>
</p>



In [17]:
# ASSUMPTION 1: 0 seats flights are not passenger flights and 
#               are removed from the analysis data.
# ASSUMPTION 2: for a route in a month, only if the proportion of passengers and 
#               seats is 100% then all the flights are 100% otherwise there are
#               not full flights.

# MAIN PROCESS:
# file reader       -> dataframe using pandas and return a list.
# map function      -> list of [(city, (flights,full)), (city, (flights,not full))
#                              , ...] 
# shuffle function  -> list of [[(city1, [(flights,full),(flights,full),..])
#                               ,(city2, [(flights,full),(flights,full),..])
#                               ,...].
# reduce function   -> list of [[(city1, averageFull)
#                               ,(city2, averageFull)
#                               ,...]..
# _____
# detailed documentation of each function below:

import pandas as pd

# prerrequisites: this reader uses pandas library to read csv and choose
#                 the relevant data.
# input:  the finename to be read with its route.
# logic:  this function reads the csv file with pandas, calculates the
#         proportion and returns it as a list including the origin city
#         and the number of flights.
# output: a list containing the percentage of capacity that was used from
#         each flight.
def file_reader_8(filename):
  df = pd.read_csv(filename)
  df = df[df['Seats']>0] 
  df['proportion'] = df['Passengers']/df['Seats']
  df = df[['Origin_city','Flights','proportion']]
  response = df.values.tolist()

  return response

# input:  the list with the proportion of the flights with origin city and
#         number of flights.
# logic:  this function will create the pairs (origin_city, (flights, 1)) when
#         the flights were full or (origin_city, (flights, 0)) when the flights
#         were not full
# output: list of tuples with (origin_city, (flights, 0)) or 
#         (origin_city, (flights, 1)) depending on the logic described.
def map_avg_flights(array):
  pairs=[]

  for l in array :
    if (l[2] == 1) :
      pairs.append((l[0],(l[1],1)))
    else :
      pairs.append((l[0],(l[1],0)))
  return (pairs)

# input:  the list with the tuples (city, (flights, full))) of the flights.
# logic:  this function will use the input and organize it in the following
#         way [[(city1, [(flights,full),(flights,full),..])
#               ,(city2, [(flights,full),(flights,full),..])
#               ,...].
#         for each city it will containt the tuples (flights, full) to be able
#         to compute the average.
# output: list with the structure in the logic.
def shuffle_avg_flights(array):

  lista = list(set([item[0] for item in array]))

  shuffled = []

  for departure in lista:
    depart = []
    for item in array:
      if departure == item[0]:
        depart.append(item[1])
    shuffled.append((departure, depart))
  
  return shuffled


# input:  the list with this structure [[(city1, [(flights,full),(flights,full),..])
#               ,(city2, [(flights,full),(flights,full),..])
#               ,...].
# logic:  for each city the function will sum the number of fligts and the
#         number of full flights (by multiplying flights * full) and compute
#         the average.
# output: list with the average of full flights per city.
def reduce_avg_flights(array):

  reduced = []

  for departure in array:
    numerator = 0
    denominator = 0
    for item in departure[1]:
      numerator += item[0]*item[1]
      denominator += item[0]
    reduced.append((departure[0],float(str(round(numerator/denominator,2)))))
  return reduced

# Function exectution
flights = file_reader_8('/content/small_file.csv')
mapped = map_avg_flights(flights)
shuffled = shuffle_avg_flights(mapped)
reduced = reduce_avg_flights(shuffled)
reduced

[('Midland, TX', 0.0),
 ('Yakima, WA', 0.05),
 ('Great Falls, MT', 0.06),
 ('Wichita, KS', 0.03),
 ('Oakland, CA', 0.0),
 ('Omaha, NE', 0.03),
 ('Butte, MT', 0.06),
 ('Jackson, MS', 0.0),
 ('Fresno, CA', 0.01),
 ('Helena, MT', 0.0),
 ('Eureka, CA', 0.0),
 ('Grand Junction, CO', 0.05),
 ('Casper, WY', 0.03),
 ('Minneapolis, MN', 0.08),
 ('Corpus Christi, TX', 0.03),
 ('Flint, MI', 0.0),
 ('Chicago, IL', 0.02),
 ('Santa Rosa, CA', 0.0),
 ('Cheyenne, WY', 0.02),
 ('Fort Lauderdale, FL', 0.0),
 ('Enid, OK', 0.0),
 ('Merced, CA', 0.0),
 ('Amarillo, TX', 0.04),
 ('Fort Dodge, IA', 0.08),
 ('La Crosse, WI', 0.23),
 ('Elko, NV', 0.02),
 ('Fayetteville, AR', 0.0),
 ('San Jose, CA', 0.0),
 ('Wichita Falls, TX', 0.06),
 ('Bakersfield, CA', 0.0),
 ('Anchorage, AK', 0.0),
 ('Fargo, ND', 0.15),
 ('Salt Lake City, UT', 0.0),
 ('Medford, OR', 0.02),
 ('Pierre, SD', 0.0),
 ('Rapid City, SD', 0.05),
 ('Garden City, KS', 0.0),
 ('Tulsa, OK', 0.04),
 ('Stockton, CA', 0.0),
 ('Pittsburgh, PA', 0.0),
 ('San

Look at your code. Can you use a <code>combine()</code> operation? 

If you cannot directly think about how you can represent the key-value pairs in a way that allows you to take advantage of a combine.

In [18]:
# ASSUMPTION 1: 0 seats flights are not passenger flights and 
#               are removed from the analysis data.
# ASSUMPTION 2: for a route in a month, only if the proportion of passengers and 
#               seats is 100% then all the flights are 100% otherwise there are
#               not full flights.

# MAIN PROCESS:
# file reader       -> dataframe using pandas and return a list.
# map function      -> list of [(city, (flights,full)), (city, (flights,not full))
#                              , ...] 
# combine function  -> list of [(city, (numerator, denominator)),
#                               (city, (numberator, denominator))
#                              , ...] 
# shuffle function  -> list of [[(city1, [(num, denom),(num, denom),..])
#                               ,(city2, [(num, denom),(num, denom),..])
#                               ,...].
# reduce function   -> list of [[(city1, averageFull)
#                               ,(city2, averageFull)
#                               ,...]..
# _____
# detailed documentation of each function below:

import pandas as pd

# prerrequisites: this reader uses pandas library to read csv and choose
#                 the relevant data.
# input:  the finename to be read with its route.
# logic:  this function reads the csv file with pandas, calculates the
#         proportion and returns it as a list including the origin city
#         and the number of flights.
# output: a list containing the percentage of capacity that was used from
#         each flight.
def file_reader_9(filename):
  df = pd.read_csv(filename)
  df = df[df['Seats']>0] 
  df['proportion'] = df['Passengers']/df['Seats']
  df = df[['Origin_city','Flights','proportion']]
  response = df.values.tolist()

  return response

# input:  the list with the proportion of the flights with origin city and
#         number of flights.
# logic:  this function will create the pairs (origin_city, (flights, 1)) when
#         the flights were full or (origin_city, (flights, 0)) when the flights
#         were not full
# output: list of tuples with (origin_city, (flights, 0)) or 
#         (origin_city, (flights, 1)) depending on the logic described.
def map_avg_flights_2(array):
  pairs=[]

  for l in array :
    if (l[2] == 1) :
      pairs.append((l[0],(l[1],1)))
    else :
      pairs.append((l[0],(l[1],0)))
  return (pairs)

# input:  the list with the tuples (city, (flights, full))) of the flights.
# logic:  this function will use the input and organize it in the following
#         way [[(city1, [(num,denom),(num,denom),..])
#               ,(city2, [(num,denom),(num,denom),..])
#               ,...].
#         for each city it will containt the tuples (num, dem) to be able
#         to compute the average.
# output: list with the structure in the logic.
def combine_avg_flights_2(array):
 
  combined = []
  for item in array:
    numerator = item[1][0]*item[1][1]
    denominator = item[1][0]
    combined.append((item[0],(numerator,denominator)))
  
  return combined


# input:  the list with the tuples (city, (num, denom))) of the flights.
# logic:  this function will use the input and organize it in the following
#         way [[(city1, [(num,denom),(num,denom),..])
#               ,(city2, [(num,denom),(num,denom),..])
#               ,...].
#         for each city it will containt the tuples (num,denom),(num,denom) 
#         to be able to compute the average.
# output: list with the structure in the logic.
def shuffle_avg_flights_2(array):

  lista = list(set([item[0] for item in array]))

  shuffled = []

  for departure in lista:
    depart = []
    for item in array:
      if departure == item[0]:
        depart.append(item[1])
    shuffled.append((departure, depart))
  
  return shuffled

# input:  the list with this structure [[(city1, [(num,denom),(num,denom),..])
#               ,(city2, [(num,denom),(num,denom),..])
#               ,...].
# logic:  for each city the function will sum the number of fligts and the
#         number of full flights (by multiplying flights * full) and compute
#         the average.
# output: list with the average of full flights per city.
def reduce_avg_flights_2(array):

  reduced = []

  for departure in array:
    numerator = 0
    denominator = 0
    for item in departure[1]:
      numerator += item[0]
      denominator += item[1]
    reduced.append((departure[0],float(str(round(numerator/denominator,2)))))
  return reduced

# Function exectution
flights = file_reader_9('/content/small_file.csv')
mapped = map_avg_flights_2(flights)
combined = combine_avg_flights_2(mapped)
shuffled = shuffle_avg_flights_2(combined)
reduced = reduce_avg_flights_2(shuffled)
reduced


[('Midland, TX', 0.0),
 ('Yakima, WA', 0.05),
 ('Great Falls, MT', 0.06),
 ('Wichita, KS', 0.03),
 ('Oakland, CA', 0.0),
 ('Omaha, NE', 0.03),
 ('Butte, MT', 0.06),
 ('Jackson, MS', 0.0),
 ('Fresno, CA', 0.01),
 ('Helena, MT', 0.0),
 ('Eureka, CA', 0.0),
 ('Grand Junction, CO', 0.05),
 ('Casper, WY', 0.03),
 ('Minneapolis, MN', 0.08),
 ('Corpus Christi, TX', 0.03),
 ('Flint, MI', 0.0),
 ('Chicago, IL', 0.02),
 ('Santa Rosa, CA', 0.0),
 ('Cheyenne, WY', 0.02),
 ('Fort Lauderdale, FL', 0.0),
 ('Enid, OK', 0.0),
 ('Merced, CA', 0.0),
 ('Amarillo, TX', 0.04),
 ('Fort Dodge, IA', 0.08),
 ('La Crosse, WI', 0.23),
 ('Elko, NV', 0.02),
 ('Fayetteville, AR', 0.0),
 ('San Jose, CA', 0.0),
 ('Wichita Falls, TX', 0.06),
 ('Bakersfield, CA', 0.0),
 ('Anchorage, AK', 0.0),
 ('Fargo, ND', 0.15),
 ('Salt Lake City, UT', 0.0),
 ('Medford, OR', 0.02),
 ('Pierre, SD', 0.0),
 ('Rapid City, SD', 0.05),
 ('Garden City, KS', 0.0),
 ('Tulsa, OK', 0.04),
 ('Stockton, CA', 0.0),
 ('Pittsburgh, PA', 0.0),
 ('San


### <strong> Bonus - Data Analysis </strong>
#### Data analysis

<p align="justify">
<font size="3">
Look at the number of almost empty flights and full flights and propose an analysis that explores if there is a correlation between the number of flights connecting two cities, the number of full flights and the percentage of almost empty flights. 

The granularity of the analysis is important: consider for example the week, the month and the year.
</font>
</p>


In [19]:
# ASSUMPTION 1: 0 seats flights are not passenger flights and 
#               are removed from the analysis data.
# ASSUMPTION 2: for a route in a month, only if the proportion of passengers and 
#               seats is 100% then all the flights are 100% otherwise there are
#               not full flights.
# ASSUMPTION 3: for a route in a month, if the proportion of passengers and 
#               seats is less or equal than 10% then all the flights are
#               assumed to be almost empty 

# MAIN PROCESS:
# file reader       -> dataframe using pandas and return a list.
# map function      -> list of [(month, (connection, (f,full,empty))), 
#                               (month, (connection, (f,full,empty))), ...] 
# shuffle function  -> list of [(month1, [(connection1, [(f,full,empty),
#                                                        (f,full,empty),...),
#                                         (connection2, [(f,full,empty),
#                                                       (f,full,empty),...),...]
#                               (month2, [(connection1, [(f,full,empty),
#                                                        (f,full,empty),...),
#                                         (connection2, [(f,full,empty),
#.                                                       (f,full,empty),...),...]
# reduce function   -> list of [(month1, [(connection1, (f,% full,% empty)),
#                                         (connection2, (f,% full,% empty)),...]
#                               (month2, [(connection1, (f,% full,% empty)),
#                                         (connection2, (f,% full,% empty)),...]
# analyze function   -> computes the correlation matrix for f, %full and %empty.
# _____
# detailed documentation of each function below:

import pandas as pd
import numpy as np

# prerrequisites: this reader uses pandas library to read csv and choose
#                 the relevant data.
# input:  the finename to be read with its route.
# logic:  this function reads the csv file with pandas, calculates the
#         proportion and returns it as a list including the origin city
#         and the number of flights.
# output: a list containing the percentage of capacity that was used from
#         each flight.
def file_reader_10(filename):
  df = pd.read_csv(filename)
  df = df[df['Seats']>0] 
  df['almost_empty'] = np.where( df['Passengers']/df['Seats'] <= 0.1, 1, 0 )
  df['full_flights'] = np.where( df['Passengers']/df['Seats'] == 1, 1, 0 )
  df['Origin'] = np.where(df['Origin_city'] < df['Destination_city'],
                          df['Origin_city'], df['Destination_city'])
  df['Destination'] = np.where(df['Origin_city'] < df['Destination_city'],
                               df['Destination_city'], df['Origin_city'])
  
  df = df[['Fly_date','Origin','Destination','Flights','almost_empty',
           'full_flights']]
  response = df.values.tolist()

  return response

# input:  the list with the proportion of the flights with origin city and
#         number of flights.
# logic:  this function will create the pairs (date( connection,
#                                                  (flights, full, empty )))
# output: list of tuples with the the logic described.
def map_corr(array):
  tuples = []
  for flight in array:
    tuples.append((flight[0],((flight[1]+' <--> '+flight[2]),
                              ((flight[3],flight[4],flight[5])))))
  
  return (tuples)


# input:  the list with the tuples (date( connection, (flights, full, empty )))
#         of the flights.
# logic:  this function will use the input and organize it in the following
#         way [(month1, [(connection1, [(f,full,empty),
#                                       (f,full,empty),...),
#                        (connection2, [(f,full,empty),
#                                       (f,full,empty),...),...]
#              (month2, [(connection1, [(f,full,empty),
#                                       (f,full,empty),...),
#                        (connection2, [(f,full,empty),
#                                       (f,full,empty),...),...]
#         for each city it will containt the tuples (num,denom),(num,denom) 
#         to be able to compute the average.
# output: list with the structure in the logic.
def shuffle_corr(array):

  dates =[]
  connections = []
  for tuples in array:
    dates.append(tuples[0])
    connections.append(tuples[1][0])

  dates = sorted(list(set(dates)))
  connections = sorted(list(set(connections)))

  all = []

  for date in dates:
    dat = []
    for connection in connections:
      conn = []
      for element in array:
        if element[0] == date and element[1][0] == connection:
          almostEmpty = 0
          full = 0
          if element[1][1][1] == 1:
            almostEmpty = element[1][1][0]
          if element[1][1][2] == 1:
            full = element[1][1][0]
          conn.append((element[1][1][0],almostEmpty,full))
      if len(conn)>0:
        dat.append((connection, conn))
    
    all.append((date, dat))
    
  return all

# input:  the list with this structure [(month1, [(connection1, [(f,full,empty),
#                                                                (f,full,empty),...),
#                                                 (connection2, [(f,full,empty),
#                                                                (f,full,empty),...),...]
#                                       (month2, [(connection1, [(f,full,empty),
#                                                                (f,full,empty),...),
#                                                 (connection2, [(f,full,empty),
#                                                                (f,full,empty),...),...]
#                                        ,...].
# logic:  for each city the function will sum the number of fligts and the
#         number of full flights (by multiplying flights * full) 
#         and the (flights * empty) and compute the percentage.
#         The solution will be the following:
#         list of [(month1, [(connection1, (f,% full,% empty)),
#                                         (connection2, (f,% full,% empty)),...]
#                               (month2, [(connection1, (f,% full,% empty)),
#                                         (connection2, (f,% full,% empty)),...]
# output: list with the percentage of full flights and empty flights per city/month.
def reduce_corr(array):

  definitive = []
  
  for date in array:
    dates = []
    for conn in date[1]:
      connection = []
      flights = 0
      almost_empty = 0
      full = 0
      for values in conn[1]:
        flights += values[0]
        almost_empty += values[1]
        full += values[2]
      connection.append((conn[0],(flights,almost_empty,full)))
      dates.append(connection)
    definitive.append((date[0],dates))
  
  return definitive

# input:  list with the percentage of full flights and empty flights per city/month.
# logic:  this function will compute the correlation matrix between the 3 variables:
#         - number of flights
#         - % full flights
#         - ∞ of empty flights 
# output: correlation matrix for the 3 variables with a granularity level of 
#         city/month. 
def analyze_corr(array):

  tables = []
  for date in array:
    for conn in date[1]:
      tables.append([conn[0][1][0],conn[0][1][1],conn[0][1][2]])
  
  df = pd.DataFrame({
    'Flights': [i[0] for i in tables], 
    'Almost_empty': [i[1] for i in tables],
    'Full': [i[2] for i in tables]
    })
  
  return df.corr()


# Function exectution
flights  = file_reader_10('/content/small_file.csv')
mapped   = map_corr(flights)
shuffled = shuffle_corr(mapped)
reduced = reduce_corr(shuffled)
analyze = analyze_corr(reduced)
analyze

# The output shows that the correlation with this granularity level is small.
# Nevertheless, the most relevant correlations are:
#     1. When there are more flights for a route A->B or B->A, there are less
#        full flights.
#     2. When there are more flights for a route A->B or B->A, there are more
#        empty flights.

,Flights,Almost_empty,Full
Flights,1.000000,0.035362,-0.065596
Almost_empty,0.035362,1.000000,-0.014208
Full,-0.065596,-0.014208,1.000000
